# Imports

In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from typing import Iterable
import gym
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

from kaggle_environments import evaluate, make, utils
from kaggle_environments.utils import Struct

# Model architecture

In [ ]:
class DQN(nn.Module):

    def __init__(
        self,
        learning_rate: float,
        input_dims: Iterable,
        middle_dims: int,
        output_dims: int,
        n_actions: int
    ):
        super(DQN, self).__init__()
        self.input_layer = nn.Linear(*input_dims, middle_dims)
        self.middle_layer = nn.Linear(middle_dims, output_dims) 
        self.output_layer = nn.Linear(output_dims, n_actions) 
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss = nn.MSELoss()
        self.device = 'cpu'
        self.to(self.device)

    def forward(self, state):
        x = F.relu(self.input_layer(state))
        x = F.relu(self.middle_layer(x))
        actions = self.output_layer(x)

        return actions

# Agent

In [ ]:
class Agent:

    ROWS = 6
    COLS = 7

    def __init__(
        self,
        gamma: float,
        learning_rate: float,
        input_dims: Iterable,
        batch_size: int,
        n_actions: int,
        max_mem_size: int = 100000
    ):
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.action_space = [i for i in range(n_actions)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_counter = 0

        self.Q_eval = DQN(
            learning_rate=self.learning_rate,
            input_dims=input_dims,
            middle_dims=128,
            output_dims=128,
            n_actions=n_actions
        )
        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=bool)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done

        self.mem_counter += 1
    
    def choose_action(self, observation: Struct, board: list) -> int:
        state = T.tensor([observation], dtype=T.float32).to(self.Q_eval.device)
        actions = self.Q_eval.forward(state)
        action = self._load_available_action(actions, board)
        return action
    
    def learn(self):
        if self.mem_counter < self.batch_size:
            return 
        
        self.Q_eval.optimizer.zero_grad()

        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem, self.batch_size, replace=False)
        batch_index = np.arange(self.batch_size, dtype=np.int32)

        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        action_batch = self.action_memory[batch]

        # loss function
        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        # q_next['terminal_batch'] = 0
        
        q_target = reward_batch + self.gamma + T.max(q_next, dim=1)[0]
        
        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()

    def _load_available_action(self, actions, board) -> int:
        board = np.array(board).reshape(self.ROWS, self.COLS).T

        base_actions_list: list = actions.tolist()[0]    
        final_actions_list: list = actions.tolist()[0]        
        actions_dict = {k: v for k, v in zip(base_actions_list, range(len(base_actions_list)))}

        for i in range(self.COLS):
            if board[i][0]:
                final_actions_list.remove(base_actions_list[i])
        
        if len(final_actions_list):
            action = actions_dict[max(final_actions_list)]
        else:
            action = 0
        
        return action


# Train agent

## Functions

In [ ]:
def get_model_input(observation: Struct) -> list:
    return observation.board + [observation.step, observation.mark]

In [ ]:
def count_reward(
    n_rounds_without_loose: int,
    baseline_reward: int
) -> int:
    reward = 0
    
    if not n_rounds_without_loose % 5:
        reward += 1
    
    if not n_rounds_without_loose % 10:
        reward += 1

    if baseline_reward == 1:
        reward = 5
    
    return reward

## Training parameters

In [ ]:
n_games = 10000
env = make("connectx", debug=True)
env.render()
trainer = env.train([None, "negamax"])

In [ ]:
agent = Agent(
    gamma=0.99,
    learning_rate=0.003,
    input_dims=[44],
    batch_size=64,
    n_actions=7
)

## Play the game!

In [ ]:
final_rewards = []

In [ ]:
start = datetime.now()

for i in range(n_games):
    done  = False
    observation = trainer.reset()
    n_rounds_without_loose = 0
    rewards = []

    while not done:
        model_input = get_model_input(observation)
        action = agent.choose_action(model_input, observation.board)
        observation_, reward, done, info = trainer.step(int(action))

        if reward != -1:
            n_rounds_without_loose += 1
        reward = count_reward(n_rounds_without_loose, reward)
        rewards.append(reward)

        model_next_input = get_model_input(observation_)
        agent.store_transition(model_input, action, reward, model_next_input, done)
        agent.learn()
        observation = observation_
    
    final_rewards.append(max(rewards))
    
stop = datetime.now()
seconds_spent = (stop - start).seconds

env.render(mode="ipython", width=500, height=500, header=False, controls=False)

# See games history

In [ ]:
def count_means(final_rewards: list) -> list:
    data_to_mean = []
    data = []

    for i in range(len(final_rewards)):
        if i % 10:
            data_to_mean.append(final_rewards[i])
        else:
            current_mean = round(np.mean(data_to_mean), 2)
            data += [current_mean]*10
            data_to_mean = []
    
    return data

In [ ]:
def load_cumulative_wins(final_rewards: list) -> list:
    data = []
    wins = 0

    for reward in final_rewards:
        if reward == 5:
            wins += 1
        
        data.append(wins)
    
    return data

In [ ]:
def show_history_plot(final_rewards: list):
    x = [i for i in range(len(final_rewards))]
    moving_means = count_means(final_rewards)
    cumulative_wins = load_cumulative_wins(final_rewards)

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scattergl(
        name='Rewards',
        x=x,
        y=final_rewards,
        mode='lines',
        opacity=0.3
    ))
    fig.add_trace(go.Scattergl(
        name='Moving mean for 10 games',
        x=x,
        y=moving_means,
        mode='lines',
        marker_color='orange'
    )) 
    fig.add_trace(go.Scattergl(
        name='Cumulative wins num',
        x=x,
        y=cumulative_wins,
        # mode='lines',
        marker_color='rgb(0, 190, 0)'
    ), secondary_y=True) 

    fig.update_layout(
        title='0: loose<br>1 or 2: 5 or 10 rounds without loose<br>5: win',
        xaxis_title='Games',
        yaxis_title='Rewards',
        yaxis2=dict(
            title="Cumulative wins",
            titlefont=dict(color='rgb(0, 190, 0)'),
            tickfont=dict(color='rgb(0, 190, 0)')
        ),
    )

    fig.show(renderer='notebook')

In [ ]:
show_history_plot(final_rewards)

In [ ]:
np.mean(final_rewards[:20])

In [ ]:
np.mean(final_rewards[-20:])

# Save weights

In [ ]:
save_path = os.path.join(os.getcwd(), 'data', 'first_models_weights')
T.save(agent.Q_eval.state_dict(), save_path)